In [1]:
import math

In [2]:
class Scaler:

  def __init__(self, data, children=(), op=None):
    self.data = data
    self._prev = set(children)
    self._op = op
    self.grad = 0


  def __add__(self, other):
    out = Scaler(self.data + other.data, children=(self, other), op='+')
    return out

  def __mul__(self, other):
    out = Scaler(self.data * other.data, children=(self, other), op='x')
    return out

  def tanh(self):
    x = self.data
    th = (math.exp(2*x) -1)/(math.exp(2*x) + 1)
    out = Scaler(th, (self,), 'tanh')
    return out

  def __str__(self):
    return f'Scaler {self.data}'

  def __repr__(self):
    return f'Scaler {self.data}'



In [3]:
a = Scaler(2)
b = Scaler(5)
c = Scaler(10)

d = a + b + c

In [90]:
e = a.tanh()
e._op

'tanh'